In [ ]:
!pip install packaging
!pip install pandas dash
!pip3 install httpx==0.20 dash plotly
!pip install pyngrok
!pip3 install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=acbaf63a0c40ac53479a32cf641030aaa784753046e60acc27dd07c041d36372
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [ ]:
# Import required packages
import pandas as pd
import plotly.express as px
import dash
from dash import dcc
from dash import html
from pyngrok import ngrok
import plotly.graph_objects as go
from dash.dependencies import Input, Output
import wget
import os

In [ ]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [ ]:
spacex_df["Launch Site"].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [ ]:
# Create a dash application
app = dash.Dash(__name__)

# Build dash app layout
app.layout = html.Div(children=[
    # Title
    html.H1(
        'SpaceX Launch Records Dashboard',
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}
    ),

    # TASK 1: Add a Launch Site Drop-down Input Component
    html.Div([
        dcc.Dropdown(
            id='site-dropdown',
            options=[
                {'label': 'All Sites', 'value': 'ALL'},
                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
            ],
            value='ALL',
            placeholder="Select a Launch Site here",
            searchable=True
        ),
    ], style={'font-size': 20, 'margin': '20px 0'}),

    html.Br(),

    # Pie Chart
    html.Div([
        dcc.Graph(id='success-pie-chart')
    ]),

    html.Br(),

    # TASK 3: Add a Range Slider to Select Payload
    html.Div([
        dcc.RangeSlider(
            id='payload-slider',
            min=0,
            max=10000,
            step=1000,
            marks={
                0: '0',
                2500: '2500',
                5000: '5000',
                7500: '7500',
                10000: '10000'
            },
            value=[0, 10000]
        )
    ], style={'margin': '20px 0'}),

    html.Br(),

    # Scatter Chart
    html.Div([
        dcc.Graph(id='success-payload-scatter-chart')
    ])
])

# TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
       Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
      filtered_df = spacex_df[spacex_df["class"] == 1].groupby("Launch Site")["class"].count().reset_index()
      fig = px.pie(filtered_df, values='class',
      names='Launch Site',
      title='Total Success Launches By Site')
      return fig
    else:
      filtered_df = spacex_df[spacex_df["Launch Site"] == entered_site].groupby("class").size().reset_index(name='Count')
      fig = px.pie(filtered_df, values='Count',
      names='class',
      title=f'Total Success Launches for site {entered_site}')
      return fig

# TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
       [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
def get_scatter_plot(entered_site, payload_range):
    min_payload, max_payload = payload_range
    filtered_df = spacex_df
    if entered_site == 'ALL':
      filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] >= min_payload) & (filtered_df['Payload Mass (kg)'] <= max_payload)]
      fig = px.scatter(filtered_df, x='Payload Mass (kg)', y="class",
      color='Booster Version',
      title='Correlation between Payload and Success for all Sites',
      labels={'category': 'Booster Version Category'})
      return fig
    else:
      filtered_df = spacex_df[spacex_df["Launch Site"] == entered_site]
      filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] >= min_payload) & (filtered_df['Payload Mass (kg)'] <= max_payload)]
      fig = px.scatter(filtered_df, x='Payload Mass (kg)', y="class",
      color='Booster Version',
      title=f'Correlation between Payload and Success for site {entered_site}',
      labels={'category': 'Booster Version Category'})
      return fig

# Authenticate ngrok
# ngrok_authtoken = "your_authtoken_here"
ngrok_authtoken = "2r0isWzl4kKSfzOPs9EgbR47yIY_7LkakyCgWKfHs5vQHxZtf"
os.system(f"ngrok authtoken {ngrok_authtoken}")

# Run the application
if __name__ == '__main__':
    public_url = ngrok.connect(8050)  # Default Dash port is 8050
    print(f"Dash app running on: {public_url}")
    app.run_server()

Dash app running on: NgrokTunnel: "https://c3d1-104-199-186-76.ngrok-free.app" -> "http://localhost:8050"


<IPython.core.display.Javascript object>

In [ ]:
spacex_df[spacex_df["class"] == 1].groupby("Launch Site")["class"].count().reset_index()

,Launch Site,class
0,CCAFS LC-40,7
1,CCAFS SLC-40,3
2,KSC LC-39A,10
3,VAFB SLC-4E,4


In [ ]:
spacex_df[spacex_df["Launch Site"] == "CCAFS LC-40"].groupby("class").size().reset_index(name='Count')

,class,Count
0,0,19
1,1,7
